In [46]:
import gmaps
from api_keys import gkey
import gmaps.geojson_geometries
import requests
import json
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

gmaps.configure(api_key=gkey)

In [29]:
zip_geojson = requests.get('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json').content
zip_geojson = json.loads(zip_geojson)

In [30]:
harris_zips = []

with open('../data/cleandata/County_Zips.json') as json_file:
    data = json.load(json_file)
    harris_zips = data['Harris']

In [31]:
len(harris_zips)

146

In [32]:
for i in harris_zips:
    i = str(i)

In [33]:
list_index = []

for i in range(len(zip_geojson['features'])):
    if zip_geojson['features'][i]['properties']['ZCTA5CE10'] not in str(harris_zips):
        list_index.append(i)
        
for index in sorted(list_index, reverse=True): 
    del zip_geojson['features'][index]
len(zip_geojson['features'])

135

In [34]:
# We will need to scale the GINI values to lie between 0 and 1
min_gini = min(harris_zips)
max_gini = max(harris_zips)
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [40]:
len(zip_geojson['features'])

135

In [42]:
colors = []
index = 0
for i in range(len(zip_geojson['features'])):
#     country_name = feature['properties']['name']
    try:
        zipcode = int(zip_geojson['features'][i]['properties']['ZCTA5CE10'])
        color = calculate_color(zipcode)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)
    index += 1

In [45]:
fig = gmaps.figure()
# countries_geojson = gmaps.geojson_geometries.load_geometry('zipcodes')
gini_layer = gmaps.geojson_layer(
    zip_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
zip_layer = gmaps.geojson_layer(zip_geojson)
fig.add_layer(zip_layer)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))